# Instalando dependências

In [21]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

In [22]:
if IN_COLAB:
    !git clone https://github.com/LucaLemos/UFRPE_AprendizagemReforco
    sys.path.append("/content/UFRPE_AprendizagemReforco")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

In [23]:
if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg
    !pip install gymnasium==1.0.0   # conferir se precisa
    #!pip install tianshou # Para criar o Replay_Buffer
    #!pip install d3rlpy==2.7.0
    # clone repository

# Criando Dataset

In [24]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
from IPython.display import clear_output
import json
import os

In [25]:
# Configurações
DATASET_SIZE = 200_000  # Tamanho do conjunto de dados (replay buffer)
LEARNING_RATE = 1e-3  # Taxa de aprendizado para o otimizador
GAMMA = 0.99  # Fator de desconto
BATCH_SIZE = 128  # Tamanho do batch para treinamento da rede neural
HIDDEN_SIZE = 128  # Tamanho da camada oculta da rede neural
EPSILON = 0.1  # Taxa de exploração (epsilon-greedy)

In [50]:
class ReplayBuffer:
    def __init__(self, capacity, batch_size, device="cpu"):
        self.buffer = deque(maxlen=capacity)
        self.capacity = capacity
        self.batch_size = batch_size
        self.device = device  # Adiciona o dispositivo (CPU/GPU)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size=None):
        if batch_size is None:
            batch_size = self.batch_size
        states, actions, rewards, next_states, dones = zip(*random.sample(self.buffer, batch_size))
        return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

    def __len__(self):
        return len(self.buffer)

    def save_config(self, file_path):
        """Salva a configuração e os dados do buffer em um arquivo JSON."""
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        
        buffer_data = {
            "buffer_size": self.capacity,
            "batch_size": self.batch_size,
            "device": self.device,
            "experience_fields": ["state", "action", "reward", "next_state", "done"],
            "memory": [
                {
                    "state": state.tolist() if isinstance(state, np.ndarray) else state,
                    "action": action,
                    "reward": reward,
                    "next_state": next_state.tolist() if isinstance(next_state, np.ndarray) else next_state,
                    "done": done
                }
                for state, action, reward, next_state, done in self.buffer
            ]
        }
        
        with open(file_path, "w") as f:
            json.dump(buffer_data, f, indent=4)

    @classmethod
    def load_config(cls, file_path):
        """Carrega a configuração e os dados do buffer de um arquivo JSON."""
        with open(file_path, "r") as f:
            buffer_data = json.load(f)
        
        buffer_size = buffer_data["buffer_size"]
        batch_size = buffer_data["batch_size"]
        device = buffer_data.get("device", "cpu")  # Padrão para 'cpu' se não estiver presente
        experiences = buffer_data["memory"]
        
        # Criar um novo ReplayBuffer
        replay_buffer = cls(buffer_size, batch_size, device)
        
        # Restaurar os dados do buffer
        for data in experiences:
            state = np.array(data["state"]) if isinstance(data["state"], list) else data["state"]
            action = data["action"]
            reward = data["reward"]
            next_state = np.array(data["next_state"]) if isinstance(data["next_state"], list) else data["next_state"]
            done = data["done"]
            replay_buffer.push(state, action, reward, next_state, done)
        
        return replay_buffer

In [51]:
# Implementação do DQNAgent
class DQNAgent:
    def __init__(self, state_dim, action_dim, lr=1e-3, gamma=0.99, hidden_size=128, device="cpu"):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.device = device

        # Rede neural para aproximar a função Q
        self.q_network = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, action_dim)
        ).to(device)

        # Otimizador
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)

    def act(self, state, epsilon=0.1):
        if random.random() < epsilon:
            return random.randint(0, self.action_dim - 1)  # Exploração
        else:
            state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device)
            with torch.no_grad():
                q_values = self.q_network(state_tensor)
            return torch.argmax(q_values).item()  # Exploração

    def update(self, batch):
        states, actions, rewards, next_states, dones = batch

        # Converte para tensores
        states = torch.tensor(np.array(states), dtype=torch.float32).to(self.device)
        actions = torch.tensor(actions, dtype=torch.long).to(self.device)
        rewards = torch.tensor(rewards, dtype=torch.float32).to(self.device)
        next_states = torch.tensor(np.array(next_states), dtype=torch.float32).to(self.device)
        dones = torch.tensor(dones, dtype=torch.float32).to(self.device)

        # Calcula os Q-values atuais
        current_q_values = self.q_network(states).gather(1, actions.unsqueeze(1))

        # Calcula os Q-values do próximo estado
        next_q_values = self.q_network(next_states).max(1)[0].detach()
        target_q_values = rewards + (1 - dones) * self.gamma * next_q_values

        # Calcula a perda (MSE)
        loss = nn.functional.mse_loss(current_q_values.squeeze(), target_q_values)

        # Backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()


In [52]:
# Função para coletar dados
def collect_data_with_dqn(env, replay_buffer, dataset_size, lr, gamma, hidden_size=128):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    agent = DQNAgent(
        state_dim=env.observation_space.shape[0],
        action_dim=env.action_space.n,
        lr=lr,
        gamma=gamma,
        hidden_size=hidden_size,
        device=device
    )

    state, _ = env.reset()
    episode_reward = 0
    episode_rewards = []

    for step in range(dataset_size):
        # Escolhe uma ação usando a política epsilon-greedy
        action = agent.act(state, epsilon=EPSILON)

        # Executa a ação no ambiente
        next_state, reward, done, truncated, info = env.step(action)

        # Armazena a transição no replay buffer
        replay_buffer.push(state, action, reward, next_state, done)

        # Atualiza o estado
        state = next_state
        episode_reward += reward

        # Se o episódio terminar, reinicia o ambiente
        if done or truncated:
            state, _ = env.reset()
            episode_rewards.append(episode_reward)
            episode_reward = 0

        # Treina o agente periodicamente
        if len(replay_buffer) > BATCH_SIZE:
            batch = replay_buffer.sample(BATCH_SIZE)
            agent.update(batch)

    return episode_rewards, agent

In [ ]:
# Passo 1: Coletar dados usando DQN
ENV_NAMES = ["CartPole-v1", "LunarLander-v3"]
ENVS_REPLAY_BUFFER = []

for env_name in ENV_NAMES:
    env = gym.make(env_name, render_mode="rgb_array")
    replay_buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE)
    episode_rewards, agent = collect_data_with_dqn(env, replay_buffer, DATASET_SIZE, LEARNING_RATE, GAMMA)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer, episode_rewards, agent))

    # Salva o replay buffer
    replay_buffer.save_config(f"config/dataset/dqn/{env_name}.json")
    print(f"Dataset gerado para {env_name} com {len(replay_buffer)} transições.")

Dataset gerado para CartPole-v1 com 200000 transições.
Dataset gerado para LunarLander-v3 com 200000 transições.


# Treinando o Modelo

## Carregando Datasets

In [54]:
import gymnasium as gym
from util.network import ReplayBuffer


In [46]:
ENV_NAMES = ["CartPole-v1", "LunarLander-v3"]

In [55]:
ENVS_REPLAY_BUFFER = []
for env_name in ENV_NAMES:
    replay_buffer = ReplayBuffer.load_config(f"config\dataset\dqn\{env_name}.json")
    env = gym.make(env_name)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer))
    

## Definições de Treino

In [178]:
ENV_NAMES = ["CartPole-v1", "LunarLander-v3"]

In [179]:
import argparse

In [180]:
def get_config(env_name, count_episodes, seed, gamma, tau, alpha, lr, steps, hidden_size=256):
    parser = argparse.ArgumentParser(description='RL')
    parser.add_argument("--run_name", type=str, default=f"{env_name}-DQN-FQI", help="Run name")
    parser.add_argument("--env", type=str, default=env_name, help="Gym environment name")
    parser.add_argument("--episodes", type=int, default=count_episodes, help="Number of episodes")
    parser.add_argument("--seed", type=int, default=seed, help="Random seed")
    parser.add_argument("--steps", type=int, default=steps, help="Training steps")
    
    parser.add_argument("--gamma", type=float, default=gamma, help="Discount factor")
    parser.add_argument("--tau", type=float, default=tau, help="Target network update rate")
    parser.add_argument("--alpha", type=float, default=alpha, help="CQL regularization weight")
    parser.add_argument("--lr", type=float, default=lr, help="Learning rate")
    parser.add_argument("--hidden_size", type=int, default=hidden_size, help="Size of hidden layers")
    
    args, _ = parser.parse_known_args()
    return args

In [181]:
import wandb
from collections import deque
from util.network import CQLAgent, save, to_one_hot
import numpy as np
import random
import torch

In [182]:
# Agente CQL
class CQLAgent:
    def __init__(self, state_size, action_size, tau, gamma, lr, alpha, model, hidden_size, device):
        self.state_size = state_size
        self.action_size = action_size
        self.tau = tau
        self.gamma = gamma
        self.lr = lr
        self.alpha = alpha
        self.device = device

        self.network = model(state_size, action_size, hidden_size).to(device)
        self.target_net = model(state_size, action_size, hidden_size).to(device)
        self.optimizer = optim.Adam(self.network.parameters(), lr=lr)

    def FQIlearn(self, experiences):
        self.network.train()
    
        states, actions, rewards, next_states, dones = experiences
    
        # Converte para tensores
        states = torch.tensor(states, dtype=torch.float32).to(self.device)
        next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
        actions = torch.tensor(actions, dtype=torch.long).to(self.device)  # Índices inteiros
        rewards = torch.tensor(rewards, dtype=torch.float32).to(self.device)
        dones = torch.tensor(dones, dtype=torch.float32).to(self.device)
    
        # Verifique a forma de actions
        if actions.dim() == 1:
            actions = actions.unsqueeze(1)  # Transforma para (batch_size, 1)
    
        with torch.no_grad():
            # Calcula Q_targets apenas com a equação de Bellman
            next_q_values = self.target_net(next_states).max(1)[0]
            q_targets = rewards + (1 - dones) * self.gamma * next_q_values
    
        # Calcula Q_values atuais
        q_values = self.network(states).gather(1, actions)  # actions já está na forma correta
    
        # Calcula a perda
        loss = nn.functional.mse_loss(q_values.squeeze(), q_targets)
    
        # Backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    
        return loss.item()


In [183]:
def train_DQN_FQI(config, buffer, model_class):
    np.random.seed(config.seed)
    random.seed(config.seed)
    torch.manual_seed(config.seed)

    env = gym.make(config.env)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    agent = CQLAgent(env.observation_space.shape[0], env.action_space.n, config.tau, config.gamma, config.lr, 
                     config.alpha, model_class, config.hidden_size, device)

    for i in range(config.steps):
        bellmann_error = agent.FQIlearn(buffer.sample())
        agent.alpha = config.alpha * np.exp(-3e-6 * i)  # Atualização do peso CQL           
        if i % 1000 == 0:
            print(f"[TRAIN {i}] Q Loss: {bellmann_error} ")
    
    save(config, save_name="FQI-DQN", model=agent.network, wandb=wandb, ep=i)
    return env

In [204]:
def evaluate(config, env, model):
    total_rewards = []
    num_episodes = config.episodes
    for i in range(num_episodes):
        print(f"[Episódio {i}]")
        state, _ = env.reset()
        done = False
        episode_reward = 0
        step = 0
        while not done:
            state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()

            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1
            if step == 10000:
                break
        total_rewards.append(episode_reward)

    print(f"Média de recompensa após {num_episodes} episódios: {sum(total_rewards) / num_episodes}")

In [185]:
# Função para extrair a tabela Q da rede neural
def extract_q_table(q_network, num_states, num_actions):
    q_table = np.zeros((num_states, num_actions))
    for state in range(num_states):
        state_tensor = torch.tensor([state], dtype=torch.long)
        state_tensor = to_one_hot(state_tensor, q_network.input_size)[0].float().unsqueeze(0)
        q_values = q_network(state_tensor).detach().numpy()
        q_table[state] = q_values
    return q_table

In [186]:
from util.qtable_helper import record_video_qtable
from util.notebook import display_videos_from_path

In [187]:

def extract_and_display(model, env_name):
    # Extrair a tabela Q da rede neural treinada
    q_table = extract_q_table(model, model.input_size, model.action_size)

    # Gravar um vídeo da política treinada
    video_path = 'videos/'  # Pasta onde os vídeos serão salvos
    video_prefix = f"fqi_{env_name}"  # Prefixo para o nome do arquivo de vídeo

    # Gravar o vídeo
    record_video_qtable(env_name, q_table, episodes=2, folder=video_path, prefix=video_prefix)

    # Exibir o vídeo gravado
    display_videos_from_path(video_path, prefix=video_prefix)

## CartPole

In [188]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [189]:
CartPole = gym.make("CartPole-v1")

In [190]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 64
STEPS = 100_000
GAMMA = 0.99  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
ALPHA = 1      # Peso do termo CQL
LEARNING_RATE = 3e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = 128

In [191]:
ENVS_REPLAY_BUFFER[0][2].batch_size = BATCH_SIZE

In [192]:
config = get_config(ENVS_REPLAY_BUFFER[0][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, LEARNING_RATE, STEPS, HIDDEN_SIZE)

In [193]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[0][2], QNetwork)

C:\Users\lucam\AppData\Local\Temp\ipykernel_19984\1777478169.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states = torch.tensor(states, dtype=torch.float32).to(self.device)
C:\Users\lucam\AppData\Local\Temp\ipykernel_19984\1777478169.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
C:\Users\lucam\AppData\Local\Temp\ipykernel_19984\1777478169.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  actions = torch.tensor(actions, dtype=torch.long).to(self.

[TRAIN 0] Q Loss: 1.4104034900665283 
[TRAIN 1000] Q Loss: 0.00010075777390738949 
[TRAIN 2000] Q Loss: 4.388205707073212e-05 
[TRAIN 3000] Q Loss: 3.053560067201033e-05 
[TRAIN 4000] Q Loss: 3.1759442208567634e-05 
[TRAIN 5000] Q Loss: 1.4607575394620653e-05 
[TRAIN 6000] Q Loss: 1.0782925528474152e-05 
[TRAIN 7000] Q Loss: 2.5532979634590447e-05 
[TRAIN 8000] Q Loss: 2.376675183768384e-05 
[TRAIN 9000] Q Loss: 2.4542772735003382e-05 
[TRAIN 10000] Q Loss: 3.270918023190461e-05 
[TRAIN 11000] Q Loss: 1.038076061377069e-05 
[TRAIN 12000] Q Loss: 2.5780833311728202e-05 
[TRAIN 13000] Q Loss: 2.992337613250129e-05 
[TRAIN 14000] Q Loss: 4.624911980499746e-06 
[TRAIN 15000] Q Loss: 0.0001906540128402412 
[TRAIN 16000] Q Loss: 6.242692961677676e-06 
[TRAIN 17000] Q Loss: 6.4689638747950085e-06 
[TRAIN 18000] Q Loss: 3.158137042191811e-05 
[TRAIN 19000] Q Loss: 1.2559566130221356e-05 
[TRAIN 20000] Q Loss: 6.218089765752666e-06 
[TRAIN 21000] Q Loss: 2.0343128198874183e-05 
[TRAIN 22000] Q 

In [194]:
import torch

In [202]:
env = ENVS_REPLAY_BUFFER[0][1]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = QNetwork(env.observation_space.shape[0], env.action_space.n, HIDDEN_SIZE).to(device)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[0][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

QNetwork(
  (fc1): Linear(in_features=4, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)

In [205]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: 9.36


In [206]:
extract_and_display(model, ENVS_REPLAY_BUFFER[0][0])

AttributeError: 'QNetwork' object has no attribute 'input_size'

## LunarLander

In [207]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [208]:
CartPole = gym.make("LunarLander-v3")

In [209]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 64
STEPS = 100_000
GAMMA = 0.99  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
ALPHA = 1      # Peso do termo CQL
LEARNING_RATE = 3e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = 128

In [210]:
ENVS_REPLAY_BUFFER[1][2].batch_size = BATCH_SIZE

In [211]:
config = get_config(ENVS_REPLAY_BUFFER[1][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, LEARNING_RATE, STEPS, HIDDEN_SIZE)

In [212]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[1][2], QNetwork)

C:\Users\lucam\AppData\Local\Temp\ipykernel_19984\1777478169.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states = torch.tensor(states, dtype=torch.float32).to(self.device)
C:\Users\lucam\AppData\Local\Temp\ipykernel_19984\1777478169.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
C:\Users\lucam\AppData\Local\Temp\ipykernel_19984\1777478169.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  actions = torch.tensor(actions, dtype=torch.long).to(self.

[TRAIN 0] Q Loss: 11.888727188110352 
[TRAIN 1000] Q Loss: 4.2599029541015625 
[TRAIN 2000] Q Loss: 6.815374851226807 
[TRAIN 3000] Q Loss: 16.661781311035156 
[TRAIN 4000] Q Loss: 159.94540405273438 
[TRAIN 5000] Q Loss: 4.9422831535339355 
[TRAIN 6000] Q Loss: 7.260927200317383 
[TRAIN 7000] Q Loss: 6.917390823364258 
[TRAIN 8000] Q Loss: 5.228265762329102 
[TRAIN 9000] Q Loss: 3.0697104930877686 
[TRAIN 10000] Q Loss: 10.500244140625 
[TRAIN 11000] Q Loss: 4.532662868499756 
[TRAIN 12000] Q Loss: 5.676116943359375 
[TRAIN 13000] Q Loss: 9.277758598327637 
[TRAIN 14000] Q Loss: 6.5929718017578125 
[TRAIN 15000] Q Loss: 6.247354507446289 
[TRAIN 16000] Q Loss: 6.98267126083374 
[TRAIN 17000] Q Loss: 164.92010498046875 
[TRAIN 18000] Q Loss: 5.616073131561279 
[TRAIN 19000] Q Loss: 6.585763931274414 
[TRAIN 20000] Q Loss: 3.6890101432800293 
[TRAIN 21000] Q Loss: 10.306714057922363 
[TRAIN 22000] Q Loss: 10.839824676513672 
[TRAIN 23000] Q Loss: 6.596324920654297 
[TRAIN 24000] Q Loss:

In [213]:
import torch

In [214]:
env = ENVS_REPLAY_BUFFER[1][1]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = QNetwork(env.observation_space.shape[0], env.action_space.n, HIDDEN_SIZE).to(device)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[1][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

QNetwork(
  (fc1): Linear(in_features=8, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
)

In [215]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: -280.402192002825


In [216]:
extract_and_display(model, ENVS_REPLAY_BUFFER[1][0])

AttributeError: 'QNetwork' object has no attribute 'input_size'